In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=ec16f2fc06bada136ae6c53857c028cf38e22bfb38392b2b4eaf18f3e7ac2fea
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

conf = SparkConf().setAppName("appName").setMaster("local[*]")
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()


In [3]:
file = spark.read.csv('movies.csv',header=True)

In [4]:
file.show()

+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|Year|Length|               Title|  Genre|               Actor|          Actress|            Director|Popularity|Awards|              Image|
+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|1990|   111|Tie Me Up! Tie Me...| Comedy|     BanderasAntonio|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1991|   113|          High Heels| Comedy|          BoséMiguel|    AbrilVictoria|      AlmodóvarPedro|        68|    No|   NicholasCage.png|
|1983|   104|        Dead ZoneThe| Horror|   WalkenChristopher|      AdamsBrooke|     CronenbergDavid|        79|    No|   NicholasCage.png|
|1979|   122|                Cuba| Action|         ConnerySean|      AdamsBrooke|       LesterRichard|         6|    No|    seanConnery.png|
|1978|    94|

In [5]:
#Question no.1
result = file.filter(file.Awards == 'Yes')
result_ = result.filter(file.Genre == 'Action')
result_final = result_.select(result_.Title , result_.Year , result_.Director)
result_final.show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



In [6]:
#Question No.2
final_result = result.select(result.Title , result.Director)
final_result.show()

+--------------------+-------------------+
|               Title|           Director|
+--------------------+-------------------+
| Fanny and Alexander|      BergmanIngmar|
|     A Man & a Woman|      LelouchClaude|
|Un Hombre y una M...|      LelouchClaude|
|   Official StoryThe|         PuenzoLuiz|
|   Wild Strawberries|      BergmanIngmar|
|Through a Glass D...|      BergmanIngmar|
|    Cries & Whispers|      BergmanIngmar|
|   Sound of MusicThe|         WiseRobert|
|     Babette's Feast|        AxelGabriel|
|Murder on the Ori...|        LumetSidney|
|     Elephant ManThe|         LynchDavid|
|E. T. The Extra-T...|    SpielbergSteven|
|              Misery|          ReinerRob|
|              Gandhi|AttenboroughRichard|
|       Autumn Sonata|      BergmanIngmar|
|            Gaslight|        CukorGeorge|
|A Woman Called Golda|         GibsonAlan|
|           Anastasia|      LitvakAnatole|
|       Cactus Flower|           SaksGene|
|             Amadeus|        FormanMilos|
+----------

In [7]:
#Question No.3
popular = file.sort(file.Popularity.desc())
popular_final = popular.filter(popular.Awards == 'No')
popular_final.select(popular_final.Title , popular_final.Popularity).show(10)

+--------------------+----------+
|               Title|Popularity|
+--------------------+----------+
|        Tango & Cash|         9|
|  Back to the Future|         9|
|Masque of the Red...|         9|
|Eighty-Four Chari...|         9|
|        Judge Priest|         9|
|           Dead-Bang|         9|
|               Glitz|         9|
|            Morituri|         9|
|Naked Gun: From t...|         9|
|        Boxing Babes|         9|
+--------------------+----------+
only showing top 10 rows



In [8]:
#Question No.4
popular = file.sort(file.Popularity.asc())
popular_final = popular.filter(popular.Year < '1980')
popular_final.select(popular_final.Title , popular_final.Popularity , popular_final.Year).show(10)

+------------------+----------+----+
|             Title|Popularity|Year|
+------------------+----------+----+
|   White Lightning|      null|1953|
|      Drop KickThe|      null|1927|
|      Desert Rider|      null|1923|
| Bank on the Stars|      null|1954|
|           Shalako|         0|1968|
|           Airport|         0|1970|
|     Anna Christie|         0|1930|
|Shout at the Devil|         0|1976|
| Tales of Tomorrow|         0|1953|
|        Indiscreet|         1|1958|
+------------------+----------+----+
only showing top 10 rows



In [11]:
#Question No.5
from pyspark.sql.functions import col
integer = file.select(col("Length").cast('int').alias('Length') , file.Genre)
avg_length = integer.groupBy('Genre').avg('Length')
avg_length.show()


+---------------+------------------+
|          Genre|       avg(Length)|
+---------------+------------------+
|          Crime|              66.0|
|        Romance|             127.0|
|      Adventure|             119.0|
|           null|             120.5|
|          Drama|113.30455259026688|
|            War|         116.90625|
|        Fantasy|             102.0|
|        Mystery|103.00990099009901|
|          Music|100.48780487804878|
|Science Fiction|106.47368421052632|
|         Horror| 93.92727272727272|
|          Short|              40.0|
|        Western|  93.0091743119266|
|         Comedy| 96.50540540540541|
|         Action|             104.5|
|       Westerns|             124.8|
+---------------+------------------+



In [13]:
#Question No.6
from pyspark.sql.functions import count
Comedy = file.filter(file.Genre == 'Comedy')
pair = Comedy.groupBy('Actor' , 'Actress').agg(count('*').alias('number'))
pairs_final = pair.filter(col('number') > 3)

pairs_final.show()

+-------------+----------------+------+
|        Actor|         Actress|number|
+-------------+----------------+------+
|ChapmanGraham|            null|     6|
| TracySpencer|HepburnKatharine|     6|
|  MurphyEddie|            null|     4|
|   CleeseJohn|            null|     4|
| SellersPeter|            null|    11|
|   AllenWoody|     KeatonDiane|     5|
|  MartinSteve|            null|     4|
|WilliamsRobin|            null|     5|
+-------------+----------------+------+



In [18]:
#Question No.7
from pyspark.sql.functions import collect_set , array_contains

Selected_Genre = file.filter(col('Genre').isin(['Comedy', 'Drama']))
Actors = Selected_Genre.groupBy('Actor').agg(collect_set('Genre').alias('genres'))

comedy_drama_actors = Actors.filter(array_contains('genres', 'Comedy') & array_contains('genres', 'Drama')).select('Actor')
comedy_drama_actors.show()


+-----------------+
|            Actor|
+-----------------+
|   AdolphsonEdvin|
|      AielloDanny|
|       AllenWoody|
|     BeattyWarren|
|BjörnstrandGunnar|
|     BoyerCharles|
|     BrandoMarlon|
|      BridgesJeff|
|        CaanJames|
|      CageNicolas|
|     CaineMichael|
|       CleeseJohn|
|      ConnerySean|
|        CruiseTom|
|       CusackJohn|
|    De NiroRobert|
|  DepardieuGérard|
|    DouglasMelvyn|
|   DouglasMichael|
|  DreyfussRichard|
+-----------------+
only showing top 20 rows



In [20]:
#Question No.8
Selected_Genre = file.filter(col('Genre').isin(['Comedy', 'Drama']))
Actors = Selected_Genre.groupBy('Actor').agg(collect_set('Genre').alias('genres'))

comedy_drama_actors = Actors.filter(array_contains('genres', 'Comedy') | array_contains('genres', 'Drama')).select('Actor')
comedy_drama_actors.show()


+------------------+
|             Actor|
+------------------+
|  AbrahamF. Murray|
|    AdolphsonEdvin|
|       AherneBrian|
|     AhlstedtBörje|
|       AielloDanny|
|         AkanTarik|
|    AlbaicínRafael|
|      AlbertEdward|
|          AldaAlan|
|        AllenWoody|
|     AlterioHector|
|     AndersonKevin|
|   AnderssonWiktor|
|AngladeJean-Hughes|
|       AnneseFrank|
|        ApfelOscar|
|     ArestrupNiels|
|         ArkinAlan|
|   ArmstrongRobert|
|     AssanteArmand|
+------------------+
only showing top 20 rows



In [25]:
#Question No.9
Comedy = file.filter(file.Genre == 'Comedy')
Actors = Comedy.select(file.Actor).distinct()
Actors_final = file.select(file.Actor).distinct().join(Actors, on='Actor', how='left_anti')

Actors_final.show()

+---------------+
|          Actor|
+---------------+
|   CottenJoseph|
|       BrownTom|
|     DillonMatt|
| ShimuraTakashi|
|   LintDerek De|
|  LancasterBurt|
|    RomeroCesar|
|  StockwellDean|
|    UrichRobert|
|       DavisGuy|
|    BridgesBeau|
|    KattWilliam|
|  EnglundRobert|
|      PriceMarc|
|CapolicchioLino|
|     FondaPeter|
| TownsendRobert|
|  ChesneyArthur|
|      BoyceAlan|
|       LoneJohn|
+---------------+
only showing top 20 rows



In [32]:
#Question No.10
from pyspark.sql.functions import mean, max, min
actor_ranking = file.groupBy('Actor').agg(mean('Popularity').alias('Mean'),max('Popularity').alias('Maximum'),min('Popularity').alias('Minimum'))

actor_ranking.show()


+------------------+-----------------+-------+-------+
|             Actor|             Mean|Maximum|Minimum|
+------------------+-----------------+-------+-------+
|              null|60.42857142857143|     77|     32|
|        AbelAlfred|             49.0|     49|     49|
|  AbrahamF. Murray|              6.0|      6|      6|
|    AdolphsonEdvin|             49.0|     72|     26|
|       AherneBrian|             57.0|     57|     57|
|     AhlstedtBörje|             81.0|     81|     81|
|       AielloDanny|             12.5|      5|     20|
|         AkanTarik|             53.0|     53|     53|
|    AlbaicínRafael|             55.0|     55|     55|
|      AlbertEdward|             82.0|     82|     82|
|          AldaAlan|             32.5|     53|     12|
|         AllenBill|             75.0|     75|     75|
|        AllenWoody|51.27272727272727|     84|     12|
|     AlterioHector|             39.0|     39|     39|
|         AmecheDon|             45.0|     45|     45|
|     Ande

In [44]:
#Question No.11
from pyspark.sql.functions import floor
decade_movies = file.withColumn('Decade', floor(file.Year / 10) * 10)
decade_count = decade_movies.groupBy('Decade').count()
decade_count = decade_count.filter(col('Decade') >= 1960)

decade_count.show()


+------+-----+
|Decade|count|
+------+-----+
|  1960|  154|
|  1970|  244|
|  1980|  607|
|  1990|  345|
+------+-----+



In [34]:
#Question No.12
year_count = file.groupBy('Year').count()
year_count.show()


+----+-----+
|Year|count|
+----+-----+
|1953|   16|
|1957|   16|
|1987|  102|
|1956|   15|
|1936|    6|
|1958|   12|
|1943|    8|
|1972|   24|
|1931|    9|
|1988|   96|
|1926|    4|
|1938|    8|
|1932|    8|
|1977|   29|
|1971|   23|
|1984|   38|
|1982|   37|
|1941|    7|
|1965|   12|
|1962|   15|
+----+-----+
only showing top 20 rows



In [42]:
#Question No.13
long_movies = file.filter(file.Length > 100)
year = long_movies.groupBy('Year', 'Genre').count()

year.show()


+----+---------------+-----+
|Year|          Genre|count|
+----+---------------+-----+
|1984|         Comedy|    4|
|1967|          Drama|    3|
|1958|          Drama|    3|
|1968|          Music|    1|
|1990|        Mystery|    4|
|1992|      Adventure|    1|
|1933|          Drama|    1|
|1982|Science Fiction|    1|
|1978|Science Fiction|    1|
|1951|            War|    1|
|1964|       Westerns|    1|
|1988|         Action|    3|
|1975|         Action|    4|
|1992|          Drama|   18|
|1969|        Western|    5|
|1951|        Mystery|    1|
|1962|        Mystery|    1|
|1950|        Mystery|    1|
|1980|        Mystery|    1|
|1990|        Western|    1|
+----+---------------+-----+
only showing top 20 rows



In [40]:
#Question No.14
movie = file.filter(file.Year < 1990)
movie_final = movie.orderBy('Title')
movie_final.show()


+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|Year|Length|               Title|          Genre|               Actor|          Actress|           Director|Popularity|Awards|              Image|
+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|1968|   139|2001: A Space Ody...|Science Fiction|          DulleaKeir|   TyzackMargaret|     KubrickStanley|        83|    No|   NicholasCage.png|
|1982|    92|             48 Hrs.|         Action|           NolteNick|   O'TooleAnnette|         HillWalter|        67|    No|   NicholasCage.png|
|1963|   138|               8 1/2|          Drama| MastroianniMarcello| CardinaleClaudia|    FelliniFederico|        80|   Yes|   NicholasCage.png|
|1966|    95|A Big Hand for th...|         Comedy|          FondaHenry|   WoodwardJoanne|        CookFielder|   

In [38]:
#Question No.15
from pyspark.sql.functions import length
long_length = file.filter(length(file.Title) > 50)
long_length.show()


+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|Year|Length|               Title| Genre|     Actor|       Actress|Director|Popularity|Awards|           Image|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|1979|    90|Fawlty TowersGour...|Comedy|CleeseJohn|ScalesPrunella|    null|        46|    No|NicholasCage.png|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+

